# MLP Example

In this notebook, we will explore how to load a custom model, specifically a Multi-Layer Perceptron (MLP), using the configuration settings defined in our project.


## Setup


### Load Local Modules


In [1]:
from pathlib import Path
import sys
import subprocess


def get_project_root() -> Path:
    return Path(
        subprocess.check_output(
            ["git", "rev-parse", "--show-toplevel"],
            cwd=".",
            stderr=subprocess.STDOUT,
        )
        .strip()
        .decode()
    )


project_root = get_project_root()
src_dir = project_root / "src"

if str(src_dir) not in sys.path:
    sys.path.append(str(src_dir))

try:
    from utils.seed_utils import set_seed
except ImportError:
    raise ImportError("Cannot import module. Make sure that the project is on the path")

SEED = 42
set_seed(SEED)

### Setup logs and config

Here's the configuration used:

```yaml
logs:
  _target_: schemas.log_schema.LogSchema
  sink: ${path_config:run_log}
  level: INFO
  format: "{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {thread.name} | {name}:{function}:{line} - {message}"
  rotation: 10 MB
  retention: 7 days
  colorize: true

model:
  _target_: models.smol_lm3.smol_lm3_model.SmolLM3Model
```

But we want to use an MLP model instead:

```yaml
model:
  _target_: models.mlp.mlp_model.MLPModel
```

So here we can override the model configuration when loading the settings, as shown below.


In [2]:
from configs.app_config import AppConfig
from utils.log_utils import setup_logs


def setup(
    config_dir: str | Path | None = None,
    config_file: str = "settings.yaml",
    overrides: list[str] | None = None,
):
    AppConfig.load(
        config_dir=config_dir,
        config_file=config_file,
        overrides=overrides,
    )
    setup_logs()


setup(overrides=["model._target_=models.mlp.mlp_model.MLPModel"])

2025-10-07 11:14:09.423 | SUCCESS  | utils.file_utils:create_dir:125 - Created directory: /home/jmaubras/OneDrive/Bureau/Pro/Aluci-Perso/aluci-ai-template/logs/20251007/111408
2025-10-07 11:14:10.093 | SUCCESS  | utils.file_utils:create_dir:125 - Created directory: /home/jmaubras/OneDrive/Bureau/Pro/Aluci-Perso/aluci-ai-template/logs/20251007/111408/configs
2025-10-07 11:14:10.437 | INFO     | models.mlp.mlp_model:load_model:66 - Loading MLPModel...
2025-10-07 11:14:10.439 | SUCCESS  | models.mlp.mlp_model:load_model:68 - MLPModel loaded and ready to use !
2025-10-07 11:14:10.439 | INFO     | models.mlp.mlp_model:__post_init__:57 - MLPModel initialized.


## Load MLP Model


In [3]:
from utils.log_utils import log

log(message=AppConfig.settings["model"].model)

2025-10-07 11:14:10.449 | DEBUG    | __main__:<module>:3 - MLP(
  (model): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=256, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=10, bias=True)
  )
)


## Here, the rest of your code. Enjoy !
